In [1]:
import numpy as py
import pandas as pd
import re
import statistics
import numpy as np
import pandas as pd
import uuid
import ssdeep
from sklearn.cluster import DBSCAN,KMeans
import Levenshtein
from pyjarowinkler import distance
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import time

/Users/n.y.r/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sparse_dot_topn import awesome_cossim_topn
import re
import pandas as pd
import numpy as np
import dask.dataframe as dd
import requests
import json
from numpy import cos, sin, arcsin, sqrt
from math import radians
from datetime import datetime
import geopy
from fuzzywuzzy import fuzz 

In [3]:
df=pd.read_csv("/Users/n.y.r/Desktop/MAPC_edited.csv")

In [34]:
total_df=pd.read_csv("/Users/n.y.r/Desktop/Grouping_Rental/rental-listing-q3-2020.csv")

In [40]:
total_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158740 entries, 0 to 158739
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            0 non-null       float64
 1   location      158740 non-null  object 
 2   ask           158589 non-null  float64
 3   bedrooms      158740 non-null  int64  
 4   title         158740 non-null  object 
 5   address       0 non-null       float64
 6   posting_date  158740 non-null  object 
 7   created_at    158740 non-null  object 
 8   updated_at    158740 non-null  object 
 9   source_id     158740 non-null  int64  
 10  survey_id     158740 non-null  int64  
 11  payload       158740 non-null  object 
 12  uid           158740 non-null  object 
 13  last_seen     158740 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 18.2+ MB


# Part 0: Drop Duplicates

In [4]:
#df = df.drop_duplicates(subset='posting_date', keep="first")
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44943 entries, 0 to 44942
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           44943 non-null  int64  
 1   Unnamed: 0.1         44943 non-null  int64  
 2   Unnamed: 0.1.1       44943 non-null  int64  
 3   id                   0 non-null      float64
 4   location             44943 non-null  object 
 5   ask                  44897 non-null  float64
 6   bedrooms             44943 non-null  int64  
 7   title                44943 non-null  object 
 8   address              0 non-null      float64
 9   posting_date         44943 non-null  object 
 10  created_at           44943 non-null  object 
 11  updated_at           44943 non-null  object 
 12  source_id            44943 non-null  int64  
 13  survey_id            44943 non-null  int64  
 14  payload              44943 non-null  object 
 15  uid                  44943 non-null 

# Part 1: Subset of all columns except "title"

In [5]:
df['merged_col']=df['ask'].astype(str)+'*'+df['image_id'].astype(str)+'*'+df['location']+'*'+df['bedrooms'].astype(str)
df = df.sort_values('merged_col')
df['gid'] = (df.groupby(['merged_col']).cumcount()==0).astype(int)
df['gid'] = df['gid'].cumsum()
vc=df['gid'].value_counts()
print(vc)
gid_lists=vc[vc>=2].index.tolist()
#print (len(gid_lists))

10720    70
6096     68
8228     52
10957    52
682      40
         ..
22017     1
24064     1
9711      1
11758     1
2047      1
Name: gid, Length: 32050, dtype: int64


In [6]:
df.reset_index(inplace=True)
df.head(10)

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,address,...,uid,last_seen,image_id,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col
0,0,0,49866,49866,NaN,POINT (-71.077709 42.384666),0.0,2,30 Caldwell St,NaN,...,5464809,2020-07-22 06:53:09.981837,NaN,0.0*nan*POINT (-71.077709 42.384666)*2020-07-2...,1,False,42.384666,-71.077709,98,0.0*nan*POINT (-71.077709 42.384666)*2
1,1,1,19666,19666,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE STUDIO-1-2-3 BEDROOM APTS** CALL JERRY,NaN,...,https://boston.craigslist.org/nos/apa/d/malden...,2020-07-08 09:29:17,00000_1JokqP78FqK_50x50c,1.0*00000_1JokqP78FqK_50x50c*POINT (-71.060502...,2,False,42.429100,-71.060502,37,1.0*00000_1JokqP78FqK_50x50c*POINT (-71.060502...
2,2,6,155772,155772,NaN,POINT (-70.875903 42.5608),1.0,0,Mason - Work/Live Trade/Barter,NaN,...,https://boston.craigslist.org/nos/apa/d/beverl...,2020-09-30 05:20:45.547405,00707_ibvTAyRjuYM_0pe0pe_50x50c,1.0*00707_ibvTAyRjuYM_0pe0pe_50x50c*POINT (-70...,3,False,42.560800,-70.875903,109,1.0*00707_ibvTAyRjuYM_0pe0pe_50x50c*POINT (-70...
3,3,9,45537,45537,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) - No B...",NaN,...,https://boston.craigslist.org/gbs/apa/d/boston...,2020-07-22 05:10:11.126147,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.364600,-71.058071,64,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...
4,4,14,74501,74501,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,...,https://boston.craigslist.org/gbs/apa/d/boston...,2020-08-12 04:37:15.496085,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.364600,-71.058071,64,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...
5,5,19,153657,153657,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,...,https://boston.craigslist.org/gbs/apa/d/boston...,2020-09-30 04:55:38.260908,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.364600,-71.058071,64,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...
6,7,24,143977,143977,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,...,https://boston.craigslist.org/nos/apa/d/malden...,2020-09-23 05:06:38.132362,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.429100,-71.060502,37,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...
7,8,27,157691,157691,NaN,POINT (-71.060502 42.4291),1.0,0,***WE HAVE MANY STUDIO'S-1-2-3-4 BEDROOM APT**...,NaN,...,https://boston.craigslist.org/nos/apa/d/malden...,2020-09-30 05:42:12.882287,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.429100,-71.060502,37,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...
8,6,21,157716,157716,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,...,https://boston.craigslist.org/nos/apa/d/malden...,2020-09-30 05:42:13.018572,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.429100,-71.060502,37,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...
9,9,29,157256,157256,NaN,POINT (-71.084298 42.7528),1.0,2,Apartment Needed for quiet middle aged single,NaN,...,https://boston.craigslist.org/gbs/apa/d/haverh...,2020-09-30 05:38:55.9809,00V0V_gZTRkutBZs4_0CI0t2_50x50c,1.0*00V0V_gZTRkutBZs4_0CI0t2_50x50c*POINT (-71...,6,True,42.752800,-71.084298,51,1.0*00V0V_gZTRkutBZs4_0CI0t2_50x50c*POINT (-71...


In [52]:
id_list=[]
for gid in gid_lists:
    subset_groups = df[df['gid'] == gid]
    subset_groups=subset_groups.reset_index()
    for i in range (len(subset_groups)):
        for j in range (i,len(subset_groups)):
            id1=subset_groups.at[i,'index']
            id2=subset_groups.at[j,'index']
            str1=subset_groups.at[i,'title']
            str2=subset_groups.at[j,'title']
            fuzzy_title_score = fuzz.ratio(str1, str2)
            if (fuzzy_title_score < 60):
                id_list.append(id1)
                id_list.append(id2)
df['flag']=True
for strange_id in id_list:
    df.at[strange_id,'flag']=False

In [53]:
df['flag'].value_counts()

True     42269
False     2674
Name: flag, dtype: int64

# Part 2: Grouping the location using k-means clustering

In [9]:
Lon_list = []
Lat_list = []

for Location_str in df['location']:
    coords = re.findall('\(([^)]+)', Location_str)

    lat_lon = coords[0].split(' ')
    lon = lat_lon[0]
    lat = lat_lon[1]
    
    Lon_list.append(lon)
    Lat_list.append(lat)
    
df['Lat'] = Lat_list
df['Lon'] = Lon_list

X=df.loc[:,['Lat','Lon']]
id_n=70
kmeans = KMeans(n_clusters=id_n, random_state=0).fit(X)
id_label=kmeans.labels_
#print (id_label)
df['loc_grp']=id_label

In [50]:
vc=df['loc_grp'].value_counts()
location_lists=vc.tolist()
print (location_lists)

[4745, 4582, 4092, 3394, 3246, 2793, 2657, 2142, 1520, 1483, 1397, 1142, 1029, 970, 901, 894, 682, 607, 590, 556, 546, 508, 430, 405, 374, 369, 271, 229, 212, 194, 192, 174, 160, 158, 135, 128, 128, 124, 121, 113, 100, 78, 75, 70, 53, 51, 45, 12, 8, 8, 7, 7, 5, 5, 4, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [51]:
np.median(location_lists)

131.5

In [10]:
df['loc_grp'].value_counts()

1     4745
16    4582
37    4092
33    3394
39    3246
      ... 
10       1
55       1
13       1
45       1
6        1
Name: loc_grp, Length: 70, dtype: int64

# Part 3: Sub_Title Grouping within Location Groups

In [11]:
list1=[x.split(" ")[0:3] for x in df['title'].tolist()]
list2=[]
for i in list1:
    clean_title=" ".join(i)
    #print (clean_title)
    #break
    list2.append(clean_title)
df['sub_title']=list2
df.head(10)

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,address,...,last_seen,image_id,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col,sub_title
0,0,0,49866,49866,NaN,POINT (-71.077709 42.384666),0.0,2,30 Caldwell St,NaN,...,2020-07-22 06:53:09.981837,NaN,0.0*nan*POINT (-71.077709 42.384666)*2020-07-2...,1,False,42.384666,-71.077709,16,0.0*nan*POINT (-71.077709 42.384666)*2,30 Caldwell St
1,1,1,19666,19666,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE STUDIO-1-2-3 BEDROOM APTS** CALL JERRY,NaN,...,2020-07-08 09:29:17,00000_1JokqP78FqK_50x50c,1.0*00000_1JokqP78FqK_50x50c*POINT (-71.060502...,2,False,42.4291,-71.060502,11,1.0*00000_1JokqP78FqK_50x50c*POINT (-71.060502...,**I HAVE STUDIO-1-2-3
2,2,6,155772,155772,NaN,POINT (-70.875903 42.5608),1.0,0,Mason - Work/Live Trade/Barter,NaN,...,2020-09-30 05:20:45.547405,00707_ibvTAyRjuYM_0pe0pe_50x50c,1.0*00707_ibvTAyRjuYM_0pe0pe_50x50c*POINT (-70...,3,False,42.5608,-70.875903,19,1.0*00707_ibvTAyRjuYM_0pe0pe_50x50c*POINT (-70...,Mason - Work/Live
3,3,9,45537,45537,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) - No B...",NaN,...,2020-07-22 05:10:11.126147,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.3646,-71.058071,39,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,The Michelle Lofts
4,4,14,74501,74501,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,...,2020-08-12 04:37:15.496085,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.3646,-71.058071,39,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,The Michelle Lofts
5,5,19,153657,153657,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,...,2020-09-30 04:55:38.260908,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.3646,-71.058071,39,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,The Michelle Lofts
6,7,24,143977,143977,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,...,2020-09-23 05:06:38.132362,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.4291,-71.060502,11,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,**I HAVE MANY
7,8,27,157691,157691,NaN,POINT (-71.060502 42.4291),1.0,0,***WE HAVE MANY STUDIO'S-1-2-3-4 BEDROOM APT**...,NaN,...,2020-09-30 05:42:12.882287,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.4291,-71.060502,11,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,***WE HAVE MANY
8,6,21,157716,157716,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,...,2020-09-30 05:42:13.018572,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,5,False,42.4291,-71.060502,11,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,**I HAVE MANY
9,9,29,157256,157256,NaN,POINT (-71.084298 42.7528),1.0,2,Apartment Needed for quiet middle aged single,NaN,...,2020-09-30 05:38:55.9809,00V0V_gZTRkutBZs4_0CI0t2_50x50c,1.0*00V0V_gZTRkutBZs4_0CI0t2_50x50c*POINT (-71...,6,False,42.7528,-71.084298,60,1.0*00V0V_gZTRkutBZs4_0CI0t2_50x50c*POINT (-71...,Apartment Needed for


In [12]:
df['gid2']=0
df['fuzzy_title_group']=0
result_df=pd.DataFrame()

## a) Group by "Group_by"

In [13]:
def groupby_title(sub_df):
    sub_df['gid2'] = (sub_df.groupby(['sub_title']).cumcount()==0).astype(int)
    sub_df['gid2'] = sub_df['gid2'].cumsum()
    return sub_df
    

## b) Group by "Sort_Values" or "Fuzzy Score"

In [14]:
value_sort_df = df.sort_values(['sub_title'])
value_sort_df

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,address,...,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col,sub_title,gid2,fuzzy_title_group
20083,20083,66881,14921,14921,NaN,POINT (-71.071697 42.2918),2100.0,2,! 2 Bed Split Huge Lots of Room!!! Text NOW,NaN,...,2100.0*00g0g_l1Vl1fv5tpH_0oM0x2_50x50c*POINT (...,13925,False,42.2918,-71.071697,22,2100.0*00g0g_l1Vl1fv5tpH_0oM0x2_50x50c*POINT (...,! 2 Bed,0,0
17636,17636,58422,157112,157112,NaN,POINT (-71.122097 42.400301),2000.0,1,"! Br. Apt., Davis Sq., avail. 11/01",NaN,...,2000.0*00V0V_ejZogrYlum4_05a03S_50x50c*POINT (...,12114,False,42.400301,-71.122097,37,2000.0*00V0V_ejZogrYlum4_05a03S_50x50c*POINT (...,"! Br. Apt.,",0,0
18188,18188,60378,14920,14920,NaN,POINT (-71.071697 42.2918),2000.0,1,!! 1 BED Split Text NOW!!!!,NaN,...,2000.0*00x0x_2dXrrHGghk_0oM0x2_50x50c*POINT (-...,12507,False,42.2918,-71.071697,22,2000.0*00x0x_2dXrrHGghk_0oM0x2_50x50c*POINT (-...,!! 1 BED,0,0
20510,20510,68337,42521,42521,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,...,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,14222,False,42.415211,-71.0086,30,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,!!! Gorgeous 2,0,0
20511,20511,68341,135278,135278,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,...,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,14222,False,42.415211,-71.0086,30,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,!!! Gorgeous 2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33518,33505,115829,28838,28838,NaN,POINT (-71.054792 42.316577),2700.0,3,🙌9/1: Savin Hill HUGE 3-bed 2-bath--NO FEE! Ea...,NaN,...,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,23633,True,42.316577,-71.054792,22,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,🙌9/1: Savin Hill,0,0
33509,33498,115812,28973,28973,NaN,POINT (-71.054792 42.316577),2700.0,3,🙌9/1: Savin Hill HUGE 3-bed 2-bath--NO FEE! Ea...,NaN,...,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,23633,True,42.316577,-71.054792,22,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,🙌9/1: Savin Hill,0,0
33503,33519,115894,46351,46351,NaN,POINT (-71.054792 42.316577),2700.0,3,🙌9/1: Savin Hill HUGE 3-bed 2-bath--NO FEE! Ea...,NaN,...,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,23633,True,42.316577,-71.054792,22,2700.0*01212_lJtF0Wga7t8_0oc0g8_50x50c*POINT (...,🙌9/1: Savin Hill,0,0
20248,20248,67464,100913,100913,NaN,POINT (-71.113566 42.374203),2100.0,0,🚨 * NO FEE!!! 2 MONTHS FREE!!! STEAL IN HARVAR...,NaN,...,2100.0*00q0q_eFh814OUuba_0cU09G_50x50c*POINT (...,14043,False,42.374203,-71.113566,61,2100.0*00q0q_eFh814OUuba_0cU09G_50x50c*POINT (...,🚨 * NO,0,0


In [15]:
def groupby_fuzzy(sub_df):
    i=0
    counter=1
    #sub_df=sub_df.sort_values(['sub_title'])
    while (i <(len(sub_df)-1)):
    #print (i)
        j=i+1
        title_i=sub_df.at[i,'sub_title']
        title_j=sub_df.at[j,'sub_title']
        fuzzy_score=fuzz.ratio(title_i, title_j)
        sub_df.at[i,'fuzzy_title_group']=counter
        if (fuzzy_score<70):
            counter+=1
            sub_df.at[j,'fuzzy_title_group']=counter
        else:
            sub_df.at[j,'fuzzy_title_group']=counter
        i=j 
    return sub_df

In [16]:
#for loc_grp_id in range (1,71):
    #sub_df=df[df['loc_grp']==loc_grp_id]
#df=df.reset_index()
#value_sort_df=value_sort_df.reset_index()

fuzzy_df=groupby_fuzzy(value_sort_df)
groupby_df=groupby_title(fuzzy_df)
    #result_df=pd.concat([result_df, sub_df])

In [17]:
groupby_df.head(20)

,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,address,...,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col,sub_title,gid2,fuzzy_title_group
20083,20083,66881,14921,14921,NaN,POINT (-71.071697 42.2918),2100.0,2,! 2 Bed Split Huge Lots of Room!!! Text NOW,NaN,...,2100.0*00g0g_l1Vl1fv5tpH_0oM0x2_50x50c*POINT (...,13925,False,42.2918,-71.071697,22,2100.0*00g0g_l1Vl1fv5tpH_0oM0x2_50x50c*POINT (...,! 2 Bed,1,14294
17636,17636,58422,157112,157112,NaN,POINT (-71.122097 42.400301),2000.0,1,"! Br. Apt., Davis Sq., avail. 11/01",NaN,...,2000.0*00V0V_ejZogrYlum4_05a03S_50x50c*POINT (...,12114,False,42.400301,-71.122097,37,2000.0*00V0V_ejZogrYlum4_05a03S_50x50c*POINT (...,"! Br. Apt.,",2,12441
18188,18188,60378,14920,14920,NaN,POINT (-71.071697 42.2918),2000.0,1,!! 1 BED Split Text NOW!!!!,NaN,...,2000.0*00x0x_2dXrrHGghk_0oM0x2_50x50c*POINT (-...,12507,False,42.2918,-71.071697,22,2000.0*00x0x_2dXrrHGghk_0oM0x2_50x50c*POINT (-...,!! 1 BED,3,12840
20510,20510,68337,42521,42521,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,...,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,14222,False,42.415211,-71.0086,30,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,!!! Gorgeous 2,4,14589
20511,20511,68341,135278,135278,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,...,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,14222,False,42.415211,-71.0086,30,2100.0*01717_9c0ERfZ1HST_0CI0t2_50x50c*POINT (...,!!! Gorgeous 2,4,14589
19633,19633,65331,118806,118806,NaN,POINT (-71.103751 42.271094),2100.0,2,!!!WILL GO FAST!!!,NaN,...,2100.0*00F0F_fsB0VLSdOja_0CI0lM_50x50c*POINT (...,13593,False,42.271094,-71.103751,65,2100.0*00F0F_fsB0VLSdOja_0CI0lM_50x50c*POINT (...,!!!WILL GO FAST!!!,5,13948
36759,36759,128023,775,775,NaN,POINT (-71.240587 42.367318),3000.0,4,!!NO FEE!! Large 4-br with granite kitchen & l...,NaN,...,3000.0*00303_7XRWZoGgPAD_0gv0co_50x50c*POINT (...,26017,False,42.367318,-71.240587,53,3000.0*00303_7XRWZoGgPAD_0gv0co_50x50c*POINT (...,!!NO FEE!! Large,6,26629
33018,33018,113972,45894,45894,NaN,POINT (-71.146645 42.345697),2700.0,0,!!NO FEE!! Large 3-br with granite kitchen & l...,NaN,...,2700.0*00P0P_2eVXvnZSaia_0CI0t2_50x50c*POINT (...,23281,False,42.345697,-71.146645,62,2700.0*00P0P_2eVXvnZSaia_0CI0t2_50x50c*POINT (...,!!NO FEE!! Large,6,23811
5638,5639,17573,152157,152157,NaN,POINT (-71.285475 42.209578),1600.0,1,!Affordable 1 bedroom in Beautiful Concord (El...,NaN,...,1600.0*00O0O_cPkNadfJx6b_07r05B_50x50c*POINT (...,3847,False,42.209578,-71.285475,42,1600.0*00O0O_cPkNadfJx6b_07r05B_50x50c*POINT (...,!Affordable 1 bedroom,7,3942
5637,5646,17589,152248,152248,NaN,POINT (-71.285475 42.209578),1600.0,1,!Affordable 1 bedroom in Beautiful Concord (El...,NaN,...,1600.0*00O0O_cPkNadfJx6b_07r05B_50x50c*POINT (...,3847,False,42.209578,-71.285475,42,1600.0*00O0O_cPkNadfJx6b_07r05B_50x50c*POINT (...,!Affordable 1 bedroom,7,3942


In [18]:
fuzzy_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44943 entries, 20083 to 25829
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                44943 non-null  int64  
 1   Unnamed: 0           44943 non-null  int64  
 2   Unnamed: 0.1         44943 non-null  int64  
 3   Unnamed: 0.1.1       44943 non-null  int64  
 4   id                   0 non-null      float64
 5   location             44943 non-null  object 
 6   ask                  44897 non-null  float64
 7   bedrooms             44943 non-null  int64  
 8   title                44943 non-null  object 
 9   address              0 non-null      float64
 10  posting_date         44943 non-null  object 
 11  created_at           44943 non-null  object 
 12  updated_at           44943 non-null  object 
 13  source_id            44943 non-null  int64  
 14  survey_id            44943 non-null  int64  
 15  payload              44943 non-n

In [25]:
fuzzy_title=fuzzy_df['fuzzy_title_group'].value_counts()
fuzzy_df['fuzzy_title_group'].value_counts()
#fuzzy_title.plot(kind='pie')

6194     41
701      40
24693    39
20482    37
10973    35
         ..
24032     1
9679      1
11726     1
13773     1
2047      1
Name: fuzzy_title_group, Length: 32791, dtype: int64

In [26]:
fuzzy_df['gid2'].value_counts()

14693    310
13028    162
15796    151
13080    144
14276    137
        ... 
1740       1
3789       1
5838       1
9960       1
2047       1
Name: gid2, Length: 20286, dtype: int64

In [63]:
fuzzy_df.to_csv('/Users/n.y.r/Desktop/MAPC_5_5.CSV')

In [61]:
temp=fuzzy_df['gid'].value_counts()
gid1_lists=temp.index.tolist()
counter=0
for gid1 in gid1_lists:
    subset_groups = fuzzy_df[fuzzy_df['gid'] == gid1]
    #print (gid2)
    subset_groups=subset_groups.reset_index()

    first_id=subset_groups.at[0,'fuzzy_title_group']
    #print (first_id)
    for i in range (1, len(subset_groups)):
        #print (subset_groups.at[i,'fuzzy_title_group'])
        if ( subset_groups.at[i,'fuzzy_title_group'] !=first_id):
            counter+=1
    #print ("***********************")
print (counter)

2836


In [57]:
co_mat = pd.crosstab(fuzzy_df.gid, fuzzy_df.fuzzy_title_group)
co_mat

fuzzy_title_group,1,2,3,4,5,6,7,8,9,10,...,32782,32783,32784,32785,32786,32787,32788,32789,32790,32791
gid,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32046,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
32047,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
32048,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [62]:
co_mat.sum()

fuzzy_title_group
1        1
2        1
3        1
4        3
5        3
        ..
32787    1
32788    1
32789    1
32790    3
32791    1
Length: 32791, dtype: int64

In [21]:
sub_df['gid2'] = (sub_df.groupby(['sub_title']).cumcount()==0).astype(int)
sub_df['gid2'] = sub_df['gid2'].cumsum()
vc2=sub_df['gid2'].value_counts()
print(vc2)
gid_lists=vc[vc>=2].index.tolist()
#print (len(gid_lists))

52     10
51      6
55      6
53      5
84      5
       ..
48      1
47      1
46      1
44      1
148     1
Name: gid2, Length: 148, dtype: int64


/Users/n.y.r/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/n.y.r/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
sub_df.head(10)

,level_0,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,...,image_id,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col,sub_title,gid2
0,863,863,2671,10224,10224,NaN,POINT (-71.046131 42.165298),1200.0,1,$1200 1 Bedroom,...,00303_ac3gjSXxkbf_0t20lM_50x50c,1200.0*00303_ac3gjSXxkbf_0t20lM_50x50c*POINT (...,536,False,42.165298,-71.046131,12,1200.0*00303_ac3gjSXxkbf_0t20lM_50x50c*POINT (...,$1200 1 Bedroom,6
1,3285,3285,9854,71931,71931,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24
2,3284,3284,9850,71932,71932,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24
3,3283,3283,9845,85607,85607,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24
4,5147,5142,15856,41354,41354,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-lit Groun...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52
5,5146,5143,15861,11554,11554,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52
6,5145,5141,15851,11556,11556,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-lit Groun...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52
7,5144,5145,15872,54399,54399,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52
8,5143,5146,15878,41350,41350,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52
9,5142,5147,15882,71935,71935,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,00N0N_6NwIdB2Wsr2_0CI0t2_50x50c,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52


In [53]:
#for x in sub_df['sub_title']:
    #for y in sub_df[]
    #print (x)
#sub_df=sub_df.reset_index()
sub_df['fuzzy_title_group']=0
i=0
counter=1
while (i <(len(sub_df)-1)):
    #print (i)
    j=i+1
    #print (j)
    title_i=sub_df.at[i,'sub_title']
    title_j=sub_df.at[j,'sub_title']
    fuzzy_score=fuzz.ratio(title_i, title_j)
    sub_df.at[i,'fuzzy_title_group']=counter
    if (fuzzy_score<70):
        counter+=1
        sub_df.at[j,'fuzzy_title_group']=counter
    else:
        sub_df.at[j,'fuzzy_title_group']=counter
    i=j    

In [54]:
sub_df.head(20)

,level_0,index,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,location,ask,bedrooms,title,...,merge_without_title,gid,flag,Lat,Lon,loc_grp,merged_col,sub_title,gid2,fuzzy_title_group
0,863,863,2671,10224,10224,NaN,POINT (-71.046131 42.165298),1200.0,1,$1200 1 Bedroom,...,1200.0*00303_ac3gjSXxkbf_0t20lM_50x50c*POINT (...,536,False,42.165298,-71.046131,12,1200.0*00303_ac3gjSXxkbf_0t20lM_50x50c*POINT (...,$1200 1 Bedroom,6,1
1,3285,3285,9854,71931,71931,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24,2
2,3284,3284,9850,71932,71932,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24,2
3,3283,3283,9845,85607,85607,NaN,POINT (-70.991668 42.15786),1495.0,2,$1495 Holbrook Crossing Spacious Sun-filled Gr...,...,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,2219,False,42.15786,-70.991668,12,1495.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1495 Holbrook Crossing,24,2
4,5147,5142,15856,41354,41354,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-lit Groun...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2
5,5146,5143,15861,11554,11554,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2
6,5145,5141,15851,11556,11556,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-lit Groun...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2
7,5144,5145,15872,54399,54399,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2
8,5143,5146,15878,41350,41350,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2
9,5142,5147,15882,71935,71935,NaN,POINT (-70.991668 42.15786),1595.0,2,$1595 Holbrook Crossing Spacious Sun-filled Gr...,...,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,3517,False,42.15786,-70.991668,12,1595.0*00N0N_6NwIdB2Wsr2_0CI0t2_50x50c*POINT (...,$1595 Holbrook Crossing,52,2


In [48]:
sub_df.to_csv('/Users/n.y.r/Desktop/test_mapc.csv ')

In [ ]:
def TF_IDF_Grouper(df, column_name):
    # Instaniate our lookup hash table
    group_lookup = {}


    # Write a function for cleaning strings and returning an array of ngrams
    def ngrams_analyzer(string):
        string = re.sub(r'[,-./]', r'', string)
        ngrams = zip(*[string[i:] for i in range(2)])  # N-Gram length is 5
        return [''.join(ngram) for ngram in ngrams]


    def find_group(row, col):
        # If either the row or the col string have already been given
        # a group, return that group. Otherwise return none
        if row in group_lookup:
            return group_lookup[row]
        elif col in group_lookup:
            return group_lookup[col]
        else:
            return None


    def add_vals_to_lookup(group, row, col):
        # Once we know the group name, set it as the value
        # for both strings in the group_lookup
        group_lookup[row] = group
        group_lookup[col] = group


    def add_pair_to_lookup(row, col):
        # in this function we'll add both the row and the col to the lookup
        group = find_group(row, col)  # first, see if one has already been added
        if group is not None:
            # if we already know the group, make sure both row and col are in lookup
            add_vals_to_lookup(group, row, col)
        else:
            # if we get here, we need to add a new group.
            # The name is arbitrary, so just make it the row
            add_vals_to_lookup(row, row, col)


    # Construct your vectorizer for building the TF-IDF matrix
    vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

    # Grab the column you'd like to group, filter out duplicate values
    # and make sure the values are Unicode
    vals = df[column_name].unique().astype('U')

    # Build the matrix!!!
    tfidf_matrix = vectorizer.fit_transform(vals)

    cosine_matrix = awesome_cossim_topn(tfidf_matrix, tfidf_matrix.transpose(), vals.size, 0.6)

    # Build a coordinate matrix
    coo_matrix = cosine_matrix.tocoo()

    # for each row and column in coo_matrix
    # if they're not the same string add them to the group lookup
    for row, col in zip(coo_matrix.row, coo_matrix.col):
        if row != col:
            add_pair_to_lookup(vals[row], vals[col])

    df['group_title_within_location'] = df[column_name].map(group_lookup).fillna(df[column_name])
    return df



In [ ]:
df['gtitle'] = (df.groupby(['title']).cumcount()==0)
df['gtitle'] = df['gtitle'].cumsum()
df['gtitle'].value_counts()
#gtitle_lists=vc1[vc1>=2].index.tolist()
#print (gtitle_lists)
#df.head(10)


In [21]:
sub_df=df[df['gtitle']==5528]
sub_df.head(10)

,Unnamed: 0,Unnamed: 0.1,id,location,ask,bedrooms,title,address,posting_date,created_at,...,uid,last_seen,image_id,merge_without_title,gid,flag,Lat,Lon,loc_grp,gtitle
28025,20496,20496,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-13 11:09:44,2020-07-08 13:30:36,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-08 09:30:36,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8746,False,42.708001,-71.163802,13,5528
28028,20138,20138,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-14 12:45:18,2020-07-08 13:30:36,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-08 09:30:36,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8747,False,42.708001,-71.163802,13,5528
28032,32751,32751,NaN,POINT (-71.163802 42.708001),1710.0,2,Pacific Mill: The Ultimate Urban Loft Communit...,NaN,2020-06-15 10:31:33,2020-07-15 04:50:58.750572,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-15 04:50:58.749964,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8748,False,42.708001,-71.163802,13,5528
28036,32750,32750,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-15 10:34:00,2020-07-15 04:50:58.745205,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-15 04:50:58.744519,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8749,False,42.708001,-71.163802,13,5528
28040,2873,2873,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-16 10:30:16,2020-07-09 17:27:03,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-09 17:27:03.918465,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8750,False,42.708001,-71.163802,13,5528
28044,2871,2871,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-17 11:16:54,2020-07-09 17:27:03,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-09 17:27:03.906461,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8751,False,42.708001,-71.163802,13,5528
28048,20144,20144,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-24 14:48:57,2020-07-08 13:30:35,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-08 09:30:35,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8752,False,42.708001,-71.163802,13,5528
28053,12610,12610,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-26 11:46:07,2020-07-01 04:32:15,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-01 00:32:15,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8753,False,42.708001,-71.163802,13,5528
28058,32729,32729,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-28 11:27:56,2020-07-15 04:50:58.630796,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-15 04:50:58.630138,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8754,False,42.708001,-71.163802,13,5528
28064,42391,42391,NaN,POINT (-71.163802 42.708001),1710.0,2,"Don't Just Rent, Rent With Style at Pacific Mi...",NaN,2020-06-29 11:05:58,2020-07-22 04:33:13.811655,...,https://boston.craigslist.org/nos/apa/d/lawren...,2020-07-22 04:33:13.810961,00O0O_axyTwX9GNOr_09G07g_50x50c,1710.0*00O0O_axyTwX9GNOr_09G07g_50x50c*POINT (...,8755,False,42.708001,-71.163802,13,5528


In [22]:

list1=[x.split(" ")[0:3] for x in df['title'].tolist()]
#df['inital_split'] = list1
#df['sub_title']=",".join(df['inital_split'].astype(str))
#title_group_list = [x[0] for x in list1]
#print (list1[0:5])
list2=[]
for i in list1:
    clean_title=" ".join(i)
    #print (clean_title)
    #break
    list2.append(clean_title)
df['sub_title']=list2
df.head(10)

,Unnamed: 0,Unnamed: 0.1,id,location,ask,bedrooms,title,address,posting_date,created_at,...,last_seen,image_id,merge_without_title,gid,flag,Lat,Lon,loc_grp,gtitle,sub_title
0,49866,49866,NaN,POINT (-71.077709 42.384666),0.0,2,30 Caldwell St,NaN,2020-07-22 03:26:33,2020-07-22 06:53:09.983787,...,2020-07-22 06:53:09.981837,NaN,0.0*nan*POINT (-71.077709 42.384666)*2020-07-2...,1,False,42.384666,-71.077709,98,1,30 Caldwell St
1,19666,19666,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE STUDIO-1-2-3 BEDROOM APTS** CALL JERRY,NaN,2020-06-25 18:32:41,2020-07-08 13:29:17,...,2020-07-08 09:29:17,00000_1JokqP78FqK_50x50c,1.0*00000_1JokqP78FqK_50x50c*POINT (-71.060502...,2,False,42.4291,-71.060502,37,2,**I HAVE STUDIO-1-2-3
6,155772,155772,NaN,POINT (-70.875903 42.5608),1.0,0,Mason - Work/Live Trade/Barter,NaN,2020-09-13 17:32:23,2020-09-30 05:20:45.547998,...,2020-09-30 05:20:45.547405,00707_ibvTAyRjuYM_0pe0pe_50x50c,1.0*00707_ibvTAyRjuYM_0pe0pe_50x50c*POINT (-70...,3,False,42.5608,-70.875903,109,3,Mason - Work/Live
9,45537,45537,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) - No B...",NaN,2020-07-02 14:55:50,2020-07-22 05:10:11.126888,...,2020-07-22 05:10:11.126147,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,4,False,42.3646,-71.058071,64,4,The Michelle Lofts
14,74501,74501,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,2020-08-04 19:08:10,2020-08-12 04:37:15.496767,...,2020-08-12 04:37:15.496085,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,5,False,42.3646,-71.058071,64,5,The Michelle Lofts
19,153657,153657,NaN,POINT (-71.058071 42.3646),1.0,0,"The Michelle Lofts (Studio, 1 Br, 2 Br) -No Br...",NaN,2020-09-17 15:03:47,2020-09-30 04:55:38.261608,...,2020-09-30 04:55:38.260908,00B0B_236Mm4nHVxv_0bi07E_50x50c,1.0*00B0B_236Mm4nHVxv_0bi07E_50x50c*POINT (-71...,6,False,42.3646,-71.058071,64,5,The Michelle Lofts
21,157716,157716,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,2020-09-15 19:06:01,2020-09-30 05:42:13.019216,...,2020-09-30 05:42:13.018572,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,7,False,42.4291,-71.060502,37,6,**I HAVE MANY
24,143977,143977,NaN,POINT (-71.060502 42.4291),1.0,0,**I HAVE MANY STUDIOS~1~2~3 BEDROOM APT** MOVE...,NaN,2020-09-15 21:43:31,2020-09-23 05:06:38.13303,...,2020-09-23 05:06:38.132362,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,8,False,42.4291,-71.060502,37,7,**I HAVE MANY
27,157691,157691,NaN,POINT (-71.060502 42.4291),1.0,0,***WE HAVE MANY STUDIO'S-1-2-3-4 BEDROOM APT**...,NaN,2020-09-21 20:50:56,2020-09-30 05:42:12.882924,...,2020-09-30 05:42:12.882287,00R0R_4SYe07Q5XpC_0CI0t2_50x50c,1.0*00R0R_4SYe07Q5XpC_0CI0t2_50x50c*POINT (-71...,9,False,42.4291,-71.060502,37,8,***WE HAVE MANY
29,157256,157256,NaN,POINT (-71.084298 42.7528),1.0,2,Apartment Needed for quiet middle aged single,NaN,2020-09-01 17:00:37,2020-09-30 05:38:55.981562,...,2020-09-30 05:38:55.9809,00V0V_gZTRkutBZs4_0CI0t2_50x50c,1.0*00V0V_gZTRkutBZs4_0CI0t2_50x50c*POINT (-71...,10,True,42.7528,-71.084298,51,9,Apartment Needed for


In [23]:
df['g_title'] = (df.groupby(['sub_title']).cumcount()==0)
df['g_title'] = df['g_title'].cumsum()
df['g_title'].value_counts()

4438     68
13034    58
15465    45
5809     39
7327     37
         ..
14027     1
3788      1
7886      1
20196     1
2047      1
Name: g_title, Length: 20286, dtype: int64

In [77]:
df=df.sort_values(['sub_title'])
df.head(10)

,Unnamed: 0,id,location,ask,bedrooms,title,address,posting_date,created_at,updated_at,source_id,survey_id,payload,uid,last_seen,image_id,inital_split,sub_title,g_title
14921,14921,NaN,POINT (-71.071697 42.2918),2100.0,2,! 2 Bed Split Huge Lots of Room!!! Text NOW,NaN,2020-06-10 14:29:00,2020-07-01 04:53:29,2020-07-01 04:53:29,1,398,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-01 00:53:29,00g0g_l1Vl1fv5tpH_0oM0x2_50x50c,"[!, 2, Bed]",! 2 Bed,6857
26238,26238,NaN,POINT (-71.071697 42.2918),2100.0,2,! 2 Bed Split Huge Lots of Room!!! Text NOW,NaN,2020-06-10 14:29:00,2020-07-08 14:47:24,2020-07-08 14:47:24,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-08 10:47:24,00g0g_l1Vl1fv5tpH_0oM0x2_50x50c,"[!, 2, Bed]",! 2 Bed,7822
6941,6941,NaN,POINT (-71.071697 42.2918),2100.0,2,! 2 Bed Split Huge Lots of Room!!! Text NOW,NaN,2020-06-10 14:29:00,2020-07-09 18:08:29,2020-07-09 18:08:29,1,402,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-09 18:08:29.737431,00g0g_l1Vl1fv5tpH_0oM0x2_50x50c,"[!, 2, Bed]",! 2 Bed,4527
157112,157112,NaN,POINT (-71.122097 42.400301),2000.0,1,"! Br. Apt., Davis Sq., avail. 11/01",NaN,2020-09-28 21:21:16,2020-09-30 05:37:39.34008,2020-09-30 05:37:39.34008,1,426,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/somerv...,2020-09-30 05:37:39.339484,00V0V_ejZogrYlum4_05a03S_50x50c,"[!, Br., Apt.,]","! Br. Apt.,",21186
26237,26237,NaN,POINT (-71.071697 42.2918),2000.0,1,!! 1 BED Split Text NOW!!!!,NaN,2020-06-10 14:32:31,2020-07-08 14:47:24,2020-07-08 14:47:24,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-08 10:47:24,00x0x_2dXrrHGghk_0oM0x2_50x50c,"[!!, 1, BED]",!! 1 BED,7822
14920,14920,NaN,POINT (-71.071697 42.2918),2000.0,1,!! 1 BED Split Text NOW!!!!,NaN,2020-06-10 14:32:31,2020-07-01 04:53:29,2020-07-01 04:53:29,1,398,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-01 00:53:29,00x0x_2dXrrHGghk_0oM0x2_50x50c,"[!!, 1, BED]",!! 1 BED,6857
6939,6939,NaN,POINT (-71.071697 42.2918),2000.0,1,!! 1 BED Split Text NOW!!!!,NaN,2020-06-10 14:32:31,2020-07-09 18:08:29,2020-07-09 18:08:29,1,402,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/dorche...,2020-07-09 18:08:29.731765,00x0x_2dXrrHGghk_0oM0x2_50x50c,"[!!, 1, BED]",!! 1 BED,4525
66935,66935,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,2020-07-17 15:33:57,2020-08-05 05:15:45.281108,2020-08-05 05:15:45.281108,1,410,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/revere...,2020-08-05 05:15:45.280409,01717_9c0ERfZ1HST_0CI0t2_50x50c,"[!!!, Gorgeous, 2]",!!! Gorgeous 2,12662
135278,135278,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,2020-09-09 17:34:24,2020-09-16 04:50:54.522481,2020-09-16 04:50:54.522481,1,422,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/revere...,2020-09-16 04:50:54.521766,01717_9c0ERfZ1HST_0CI0t2_50x50c,"[!!!, Gorgeous, 2]",!!! Gorgeous 2,19263
54824,54824,NaN,POINT (-71.0086 42.415211),2100.0,2,!!! Gorgeous 2 Bedroom Apartment!!!,NaN,2020-07-17 15:33:57,2020-07-29 05:08:51.76662,2020-07-29 05:08:51.76662,1,408,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/revere...,2020-07-29 05:08:51.765932,01717_9c0ERfZ1HST_0CI0t2_50x50c,"[!!!, Gorgeous, 2]",!!! Gorgeous 2,11206


In [69]:
sub_df=df[df['g_title']==7814]
sub_df.head(10)

,Unnamed: 0,id,location,ask,bedrooms,title,address,posting_date,created_at,updated_at,source_id,survey_id,payload,uid,last_seen,image_id,inital_split,sub_title,g_title
22447,22447,NaN,POINT (-71.134159 42.393556),2500.0,2,"Renovated 2 bed/1 bath, laundry, s/s appl, dis...",NaN,2020-07-03 19:43:24,2020-07-08 14:04:34,2020-07-08 14:04:34,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:34,00m0m_921u9at0UDf_09G07g_50x50c,"[Renovated, 2, bed/1]",Renovated 2 bed/1,7814
22448,22448,NaN,POINT (-71.134159 42.393556),3700.0,4,"Contemporary 4BD, 2 Baths, 2 Parking, 6 min to...",NaN,2020-06-29 19:50:42,2020-07-08 14:04:34,2020-07-08 14:04:34,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:34,01616_kQM6ZEob3aW_08g06c_50x50c,"[Contemporary, 4BD,, 2]","Contemporary 4BD, 2",7814
22449,22449,NaN,POINT (-71.134159 42.393556),3700.0,4,Gorgeous and Sunny 4 bed steps to Red Line w/p...,NaN,2020-06-20 16:38:41,2020-07-08 14:04:34,2020-07-08 14:04:34,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:34,00j0j_cr83K3RtWYY_0ak07K_50x50c,"[Gorgeous, and, Sunny]",Gorgeous and Sunny,7814
22450,22450,NaN,POINT (-71.134159 42.393556),3700.0,4,9/1!Modern 4BD!2 Baths!6 min to T!Laundry!EIK/...,NaN,2020-06-18 22:53:12,2020-07-08 14:04:34,2020-07-08 14:04:34,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:34,00P0P_56tMn2iDjCn_0ak07K_50x50c,"[9/1!Modern, 4BD!2, Baths!6]",9/1!Modern 4BD!2 Baths!6,7814
22451,22451,NaN,POINT (-71.153468 42.378844),5000.0,5,"Amazing 5BD Townhouse, 3.5 Baths, Parking,Laun...",NaN,2020-06-29 19:46:52,2020-07-08 14:04:39,2020-07-08 14:04:39,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:39,00505_g8hrTtOMDti_50x50c,"[Amazing, 5BD, Townhouse,]","Amazing 5BD Townhouse,",7814
22452,22452,NaN,POINT (-71.153468 42.378844),5000.0,5,"Amazing 5BD Townhouse, 3.5 Baths, Parking,Laun...",NaN,2020-06-13 16:45:54,2020-07-08 14:04:39,2020-07-08 14:04:39,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:39,00505_g8hrTtOMDti_50x50c,"[Amazing, 5BD, Townhouse,]","Amazing 5BD Townhouse,",7814
22453,22453,NaN,POINT (-71.081956 42.372704),3750.0,3,Modern 3BR/1.5BA Across from Park! This Apt Ha...,NaN,2020-06-29 19:44:53,2020-07-08 14:04:39,2020-07-08 14:04:39,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:39,00W0W_15J3dUijHme_50x50c,"[Modern, 3BR/1.5BA, Across]",Modern 3BR/1.5BA Across,7814
22454,22454,NaN,POINT (-71.12308 42.393488),2150.0,1,"Great Modern 1BD, HW/ Incl, 2 min to Davis T,...",NaN,2020-06-29 19:43:10,2020-07-08 14:04:39,2020-07-08 14:04:39,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/somerv...,2020-07-08 10:04:39,00Z0Z_fIG1nmtkA2M_08g06c_50x50c,"[Great, Modern, 1BD,]","Great Modern 1BD,",7814
22455,22455,NaN,POINT (-71.091047 42.37003),2800.0,2,"Condo Quality 2BD, HT/HW/ Incl, Laundry, Moder...",NaN,2020-06-30 16:28:16,2020-07-08 14:04:45,2020-07-08 14:04:45,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:44,00p0p_4MBsQYVhYTs_05a03S_50x50c,"[Condo, Quality, 2BD,]","Condo Quality 2BD,",7814
22456,22456,NaN,POINT (-71.091047 42.37003),2800.0,2,"Condo Quality 2BD, HT/HW/ Incl, Laundry, Moder...",NaN,2020-06-29 19:38:29,2020-07-08 14:04:45,2020-07-08 14:04:45,1,400,"{""CategoryID"":1,""ImageThumb"":""https://images.c...",https://boston.craigslist.org/gbs/apa/d/cambri...,2020-07-08 10:04:45,00p0p_4MBsQYVhYTs_05a03S_50x50c,"[Condo, Quality, 2BD,]","

In [ ]:
df['sub_title']=

In [ ]:
chunk_df_list=[]
for loc_gr in id_label:
    sub_df=df[df['loc_grp']==id_label]
    grouped_df = TF_IDF_Grouper(sub_df, 'title')
    chunk_df_list.append(grouped_df)
final_df_loc_title = pd.concat(chunk_df_list)

In [ ]:
final_df_loc_title.info(verbose=True)